In [ ]:
!pip install streamlit openai sentence-transformers faiss-cpu requests beautifulsoup4 lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List, Dict, Set
import openai
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
import re
import time
from urllib.parse import urljoin, urlparse
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

In [ ]:
class ImprovedRAGSystem:
    """
    Improved RAG system with better content extraction and answer generation.
    Fixed to work properly with OpenAI API and give complete, accurate answers.
    OPTIMIZED for performance - no more hanging on chunking or embeddings.
    """

    def __init__(self):
        self.scraped_data = []
        self.chunks = []
        self.vector_store = None
        self.embedding_model = None
        self.openai_client = None
        self.is_initialized = False

        # Improved configuration
        self.BASE_URL = "https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/"
        self.CHUNK_SIZE = 800  # Larger chunks for more context
        self.CHUNK_OVERLAP = 100  # More overlap to preserve context
        self.EMBEDDING_MODEL = "all-MiniLM-L6-v2"

    def setup_openai(self, api_key: str):
        """Setup OpenAI client with API key."""
        self.openai_client = openai.OpenAI(api_key=api_key)
        print("OpenAI client initialized")

    def scrape_website(self, max_pages: int = 15):
        """
        Enhanced web scraping to get complete, relevant content.
        Focus on getting full content, especially for tuition and financial info.
        """
        print(f"🕷️ Starting enhanced scraping: {self.BASE_URL}")

        session = requests.Session()
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })

        visited_urls = set()
        urls_to_scrape = [self.BASE_URL]
        scraped_count = 0

        # Enhanced keywords - especially for tuition and financial info
        relevant_keywords = [
            'admission', 'admissions', 'apply', 'application',
            'curriculum', 'courses', 'course', 'program',
            'faculty', 'professors', 'staff',
            'tuition', 'cost', 'financial', 'aid', 'scholarship', 'funding',
            'requirements', 'prerequisite', 'career', 'outcomes',
            'employment', 'student', 'life', 'experience',
            'capstone', 'project', 'research', 'faq', 'faqs',
            'price', 'fee', 'fees', 'payment', 'billing'
        ]

        while urls_to_scrape and scraped_count < max_pages:
            current_url = urls_to_scrape.pop(0)

            if current_url in visited_urls:
                continue

            print(f"📄 Scraping: {current_url}")

            try:
                response = session.get(current_url, timeout=15)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')

                # Enhanced content extraction
                content_data = self._extract_enhanced_content(soup, current_url)
                if content_data and content_data['content'] and len(content_data['content']) > 100:
                    self.scraped_data.append(content_data)
                    scraped_count += 1
                    print(f"Extracted {content_data['length']} characters from {content_data['title']}")

                visited_urls.add(current_url)

                # Find relevant links (prioritize from main page)
                if current_url == self.BASE_URL:
                    new_links = self._find_enhanced_links(soup, self.BASE_URL, relevant_keywords)
                    for link in new_links:
                        if link not in visited_urls and link not in urls_to_scrape:
                            urls_to_scrape.append(link)
                            print(f"Found relevant link: {link}")

                time.sleep(1)  # Be respectful

            except Exception as e:
                print(f"Error scraping {current_url}: {str(e)}")
                continue

        print(f"Enhanced scraping complete! Collected {len(self.scraped_data)} pages")
        return self.scraped_data

    def _extract_enhanced_content(self, soup, url):
        """
        Enhanced content extraction that specifically preserves URLs and key structured data.
        """
        if not soup:
            return None

        # Remove unwanted elements
        for element in soup(["script", "style", "nav", "header", "footer", "aside"]):
            element.decompose()

        # Get title
        title = ""
        title_tag = soup.find('title')
        if title_tag:
            title = title_tag.get_text().strip()

        # Extract content with special attention to URLs and structured data
        content_parts = []

        # Strategy 1: Look for main content containers
        main_selectors = [
            'main', '.main-content', '#main-content', '.content',
            '.post-content', '.entry-content', '.page-content',
            '.container', '.wrapper', '.main', 'article'
        ]

        main_content = None
        for selector in main_selectors:
            main_content = soup.select_one(selector)
            if main_content:
                break

        if not main_content:
            main_content = soup.find('body')

        if main_content:
            # Extract text while preserving URLs and important formatting
            for element in main_content.find_all([
                'p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6',
                'li', 'div', 'span', 'td', 'th', 'dd', 'dt', 'a'
            ]):
                text = element.get_text().strip()

                # Special handling for links
                if element.name == 'a' and element.get('href'):
                    href = element.get('href')
                    if href.startswith('http') or href.startswith('www'):
                        text = f"{text} [URL: {href}]"

                # Keep longer text segments and those with important keywords
                if text and (len(text) > 15 or any(keyword in text.lower() for keyword in [
                    'tuition', 'cost', 'fee', 'scholarship', 'financial', 'admission',
                    'requirement', 'course', 'program', 'capstone', 'faculty',
                    'deadline', 'apply', 'contact', 'advisor', 'http', 'portal'
                ])):
                    content_parts.append(text)

        # Combine content
        content_text = " ".join(content_parts)

        # Clean up while preserving URLs
        content_text = re.sub(r'\s+', ' ', content_text)  # Normalize whitespace
        content_text = content_text.strip()

        return {
            'url': url,
            'title': title,
            'content': content_text,
            'length': len(content_text)
        }

    def _find_enhanced_links(self, soup, base_url, keywords):
        """Enhanced link finding with better keyword matching."""
        if not soup:
            return []

        relevant_links = []

        for link in soup.find_all('a', href=True):
            href = link.get('href')
            link_text = link.get_text().lower().strip()

            # Convert relative URLs to absolute
            full_url = urljoin(base_url, href)

            # Enhanced relevance checking
            is_relevant = (
                self._is_same_domain(full_url, base_url) and
                (any(keyword in link_text for keyword in keywords) or
                 any(keyword in href.lower() for keyword in keywords) or
                 'faq' in href.lower() or 'tuition' in href.lower() or
                 'cost' in href.lower() or 'financial' in href.lower())
            )

            if is_relevant:
                relevant_links.append(full_url)

        return list(set(relevant_links))

    def _is_same_domain(self, url1, url2):
        """Check if URLs are from same domain."""
        try:
            return urlparse(url1).netloc == urlparse(url2).netloc
        except:
            return False

    def create_enhanced_chunks(self):
        """
        OPTIMIZED: Enhanced chunking strategy with fast regex processing.
        No more hanging on micro-chunk creation!
        """
        print("Creating enhanced text chunks with specialized micro-chunks...")

        self.chunks = []
        total_docs = len(self.scraped_data)

        for doc_idx, data in enumerate(self.scraped_data):
            print(f"Processing document {doc_idx + 1}/{total_docs}: {data['title'][:50]}...")

            # Combine title and content with better formatting
            document = f"Page Title: {data['title']}\nSource URL: {data['url']}\n\nContent:\n{data['content']}"

            # Clean text while preserving important formatting
            cleaned_doc = re.sub(r'\s+', ' ', document)

            # Extract metadata for better source tracking
            source_url = data['url']
            title = data['title']

            # STRATEGY 1: Create specialized micro-chunks for key facts (OPTIMIZED)
            print(f"  Creating micro-chunks...")
            self._create_micro_chunks_optimized(cleaned_doc, doc_idx, source_url, title)

            # STRATEGY 2: Create regular overlapping chunks (existing logic)
            print(f"  Creating regular chunks...")
            chunk_count = 0
            for i in range(0, len(cleaned_doc), self.CHUNK_SIZE - self.CHUNK_OVERLAP):
                chunk_text = cleaned_doc[i:i + self.CHUNK_SIZE]

                # Skip very short chunks
                if len(chunk_text) < 150:
                    continue

                # Ensure chunks end at sentence boundaries when possible
                if i + self.CHUNK_SIZE < len(cleaned_doc):
                    # Try to end at a sentence
                    last_period = chunk_text.rfind('.')
                    last_question = chunk_text.rfind('?')
                    last_exclamation = chunk_text.rfind('!')

                    sentence_end = max(last_period, last_question, last_exclamation)
                    if sentence_end > len(chunk_text) * 0.8:  # If sentence end is in last 20%
                        chunk_text = chunk_text[:sentence_end + 1]

                self.chunks.append({
                    'text': chunk_text.strip(),
                    'doc_id': doc_idx,
                    'chunk_id': len(self.chunks),
                    'source_url': source_url,
                    'title': title,
                    'chunk_type': 'regular'
                })
                chunk_count += 1

            print(f"  ✓ Created {chunk_count} regular chunks from this document")

        print(f"✓ Created {len(self.chunks)} total enhanced chunks (including micro-chunks)")

    def _create_micro_chunks_optimized(self, document, doc_idx, source_url, title):
        """
        OPTIMIZED: Fast micro-chunk creation that prevents regex backtracking issues.
        """

        # Limit document size for regex processing to prevent hanging
        if len(document) > 50000:  # If document is very long, process in sections
            sections = [document[i:i+50000] for i in range(0, len(document), 45000)]
        else:
            sections = [document]

        for section in sections:
            # Simple, fast patterns (no complex quantifiers that cause backtracking)
            quick_patterns = {
                'tuition_cost': [
                    r'\$\d{1,2},?\d{3}\s*per\s*course',
                    r'\$\d{2},?\d{3}\s*total',
                    r'tuition[^.]{0,50}\$\d{1,2},?\d{3}',
                    r'\$\d{1,2},?\d{3}[^.]{0,30}tuition'
                ],
                'scholarship_names': [
                    r'Data Science Institute Scholarship',
                    r'MS in Applied Data Science Alumni Scholarship',
                    r'[A-Z][a-z]+\s+[A-Z][a-z]+\s+Scholarship'
                ],
                'deadlines': [
                    r'deadline[^.]{0,100}(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}',
                    r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}[^.]{0,50}deadline',
                    r'application[^.]{0,50}due[^.]{0,50}\d{1,2}/\d{1,2}/\d{4}'
                ],
                'urls': [
                    r'https?://[^\s<>"{}|\\\^`\[\]]{10,100}',
                    r'www\.[^\s<>"{}|\\\^`\[\]]{5,50}'
                ],
                'contact_info': [
                    r'contact[^.]{0,50}(?:Patrick|Jose)',
                    r'(?:Patrick|Jose)[^.]{0,50}enrollment',
                    r'advising[^.]{0,30}appointment'
                ]
            }

            # Process each pattern type quickly
            for info_type, patterns in quick_patterns.items():
                for pattern in patterns:
                    try:
                        # Add timeout protection and limit matches
                        matches = list(re.finditer(pattern, section, re.IGNORECASE))[:5]  # Max 5 matches per pattern

                        for match in matches:
                            # Get context around the match (±200 chars)
                            start = max(0, match.start() - 200)
                            end = min(len(section), match.end() + 200)
                            context = section[start:end].strip()

                            # Clean up context boundaries (simple version)
                            if len(context) > 100:  # Ensure meaningful context
                                # Simple boundary cleanup
                                if start > 0 and ' ' in context[:50]:
                                    space_idx = context.find(' ')
                                    context = context[space_idx:].strip()

                                self.chunks.append({
                                    'text': f"KEY {info_type.upper()}: {context}",
                                    'doc_id': doc_idx,
                                    'chunk_id': len(self.chunks),
                                    'source_url': source_url,
                                    'title': title,
                                    'chunk_type': 'micro',
                                    'info_type': info_type,
                                    'priority': 1.0
                                })

                    except re.error:
                        # Skip problematic patterns
                        continue

    def create_embeddings(self):
        """OPTIMIZED: Create vector embeddings with progress tracking."""
        print("Loading embedding model...")
        self.embedding_model = SentenceTransformer(self.EMBEDDING_MODEL)

        print(f"Creating embeddings for {len(self.chunks)} chunks...")
        texts = [chunk['text'] for chunk in self.chunks]

        # Create embeddings in smaller batches with progress tracking
        batch_size = 16  # Reduced from 32 for better memory management
        all_embeddings = []

        total_batches = (len(texts) + batch_size - 1) // batch_size

        for i in range(0, len(texts), batch_size):
            batch_num = (i // batch_size) + 1
            batch_texts = texts[i:i + batch_size]

            print(f"Processing batch {batch_num}/{total_batches} ({len(batch_texts)} chunks)...")

            # Create embeddings for this batch
            batch_embeddings = self.embedding_model.encode(
                batch_texts,
                show_progress_bar=False,  # Disable model's progress bar since we have our own
                convert_to_numpy=True
            )
            all_embeddings.append(batch_embeddings)

            print(f"✓ Batch {batch_num} complete")

        print("Combining embeddings...")
        # Combine all embeddings
        embeddings = np.vstack(all_embeddings)

        print("Creating FAISS index...")
        # Create FAISS index
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatIP(dimension)

        # Normalize for cosine similarity
        faiss.normalize_L2(embeddings)
        index.add(embeddings)

        self.vector_store = {
            'index': index,
            'embeddings': embeddings,
            'chunks': self.chunks
        }

        print(f"✓ Vector index created with {dimension}-dimensional embeddings")
        print(f"✓ Total chunks indexed: {len(self.chunks)}")

    def search_chunks(self, query: str, k: int = 8):
        """
        Enhanced search with keyword boosting and priority weighting.
        """
        if not self.vector_store:
            return []

        # Encode query
        query_embedding = self.embedding_model.encode([query])
        faiss.normalize_L2(query_embedding)

        # Search with more results initially
        scores, indices = self.vector_store['index'].search(query_embedding, min(k * 3, len(self.chunks)))

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.chunks):
                result = self.chunks[idx].copy()
                result['semantic_score'] = float(score)
                results.append(result)

        # Apply keyword boosting and priority weighting
        enhanced_results = self._apply_keyword_boosting(query, results)

        # Sort by enhanced score and return top k
        enhanced_results.sort(key=lambda x: x['final_score'], reverse=True)

        return enhanced_results[:k]

    def _apply_keyword_boosting(self, query, results):
        """
        Apply keyword boosting and priority weighting to search results.
        """
        query_lower = query.lower()

        # Define keyword boost mappings
        keyword_boosts = {
            'tuition': ['tuition', 'cost', 'fee', 'price', '$', 'dollar'],
            'scholarship': ['scholarship', 'financial aid', 'funding', 'grant'],
            'deadline': ['deadline', 'due date', 'application', 'submit', 'apply by'],
            'contact': ['contact', 'appointment', 'advisor', 'advising', 'schedule', 'meet'],
            'url': ['link', 'website', 'portal', 'apply', 'registration']
        }

        for result in results:
            text_lower = result['text'].lower()

            # Start with semantic score
            final_score = result['semantic_score']

            # Apply micro-chunk priority boost
            if result.get('chunk_type') == 'micro':
                final_score *= 1.5  # 50% boost for micro-chunks

            # Apply keyword boosting
            for category, keywords in keyword_boosts.items():
                if any(keyword in query_lower for keyword in keywords):
                    # Count keyword matches in the chunk
                    matches = sum(1 for keyword in keywords if keyword in text_lower)
                    if matches > 0:
                        final_score *= (1 + 0.2 * matches)  # Boost based on keyword frequency

            # Special boost for exact cost matches
            if any(term in query_lower for term in ['tuition', 'cost', 'price', 'fee']):
                if any(pattern in text_lower for pattern in ['$', 'dollar', 'per course', 'total']):
                    final_score *= 1.3

            # Special boost for scholarship name matches
            if 'scholarship' in query_lower:
                if any(name in text_lower for name in ['data science institute', 'alumni scholarship']):
                    final_score *= 1.4

            # Special boost for URL presence when contact/appointment mentioned
            if any(term in query_lower for term in ['contact', 'appointment', 'schedule', 'advisor']):
                if 'http' in text_lower or 'portal' in text_lower:
                    final_score *= 1.3

            result['final_score'] = final_score

        return results

    def generate_enhanced_answer(self, query: str, chunks: List[Dict]):
        """
        Enhanced answer generation with better context handling and fact extraction.
        """
        if not self.openai_client:
            return "OpenAI client not initialized"

        # Separate micro-chunks and regular chunks
        micro_chunks = [c for c in chunks if c.get('chunk_type') == 'micro']
        regular_chunks = [c for c in chunks if c.get('chunk_type') != 'micro']

        # Build context with micro-chunks prioritized
        context_parts = []

        # Add micro-chunks first (key facts)
        if micro_chunks:
            context_parts.append("KEY FACTS:")
            for i, chunk in enumerate(micro_chunks):
                context_parts.append(f"FACT {i+1}: {chunk['text']}")
            context_parts.append("\nADDITIONAL CONTEXT:")

        # Add regular chunks
        for i, chunk in enumerate(regular_chunks):
            context_parts.append(f"Source {i+1} (from {chunk['title']}):\n{chunk['text']}\n")

        context = "\n".join(context_parts)

        # Enhanced prompt with specific instructions for key facts
        prompt = f"""You are an expert assistant for the MS in Applied Data Science program at the University of Chicago.

Your task is to provide comprehensive, accurate answers based on the official program information provided below.

CONTEXT FROM OFFICIAL UCHICAGO WEBSITE:
{context}

QUESTION: {query}

CRITICAL INSTRUCTIONS:
1. **COSTS/TUITION**: If asking about costs, you MUST include exact dollar amounts (e.g., "$5,967 per course", "$71,604 total tuition")
2. **SCHOLARSHIPS**: If asking about scholarships, you MUST mention specific scholarship names like "Data Science Institute Scholarship" and "MS in Applied Data Science Alumni Scholarship"
3. **DEADLINES**: If asking about deadlines, provide ALL specific dates mentioned (format: Month Day, Year)
4. **CONTACT/APPOINTMENTS**: If asking about scheduling or advising, include any URLs or portal links mentioned
5. **EXACT QUOTES**: For key facts, use the exact wording from the source material
6. **COMPLETENESS**: Provide all relevant details, not just summaries
7. **STRUCTURE**: Use bullet points for lists (costs, deadlines, requirements)
8. **SOURCE VERIFICATION**: If information seems incomplete, state "Additional details may be available on the official website"

Based ONLY on the information provided above, give a complete and detailed answer:

ANSWER:"""

        try:
            # Use GPT-3.5-turbo with optimized parameters for factual accuracy
            response = self.openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "system",
                        "content": "You are an expert assistant for the UChicago MS in Applied Data Science program. You MUST provide complete, factual answers with exact details (costs, dates, names, URLs) from the provided context. Never summarize or generalize key facts."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                max_tokens=1000,  # More tokens for complete answers
                temperature=0.1,  # Very low temperature for maximum factual accuracy
                top_p=0.9,
                frequency_penalty=0.0,
                presence_penalty=0.0
            )
            return response.choices[0].message.content

        except Exception as e:
            error_msg = str(e)
            if "quota" in error_msg.lower() or "429" in error_msg:
                return "OpenAI API quota exceeded. Please add credits to your OpenAI account at https://platform.openai.com/account/billing"
            elif "401" in error_msg:
                return "Invalid OpenAI API key. Please check your API key."
            else:
                return f"Error generating response: {error_msg}"

    def ask_question(self, query: str):
        """Enhanced question answering with better error handling."""
        if not self.is_initialized:
            return "System not initialized. Please run full setup first."

        print(f"Searching for: {query}")

        # Search for relevant chunks with more results
        relevant_chunks = self.search_chunks(query, 5)

        if not relevant_chunks:
            return "No relevant information found."

        # Generate enhanced answer
        answer = self.generate_enhanced_answer(query, relevant_chunks)

        # Display results with better formatting
        print("\n" + "="*100)
        print("ANSWER:")
        print(answer)
        print("\n" + "="*100)
        print("SOURCES:")
        for i, chunk in enumerate(relevant_chunks):
            print(f"\nSource {i+1} (Relevance: {chunk.get('final_score', chunk.get('semantic_score', 0)):.3f}):")
            print(f"Title: {chunk['title']}")
            print(f"URL: {chunk['source_url']}")
            print(f"Content Preview: {chunk['text'][:300]}...")
            print("-" * 80)
        print("="*100)

        return answer

    def initialize_system(self, openai_api_key: str, max_pages: int = 15):
        """OPTIMIZED: Initialize the complete enhanced RAG system."""
        print("Initializing Enhanced RAG System for UChicago MS Program...")

        # Setup OpenAI
        self.setup_openai(openai_api_key)

        # Enhanced scraping
        self.scrape_website(max_pages)

        if not self.scraped_data:
            print("Failed to scrape data")
            return False

        # Create enhanced chunks (ONLY ONCE - no duplication)
        self.create_enhanced_chunks()

        # Create embeddings (optimized with progress tracking)
        self.create_embeddings()

        self.is_initialized = True
        print("✓ Enhanced RAG System fully initialized and ready!")
        return True

In [ ]:
# ENHANCED COLAB INTERFACE

def create_enhanced_interface():
    """Create an enhanced interactive interface for Google Colab."""

    # Initialize enhanced system
    rag_system = ImprovedRAGSystem()

    print("Enhanced UChicago MS Data Science Q&A Bot")

    # Get OpenAI API key with better instructions
    print("OpenAI API Key Required:")
    print("If you don't have credits, add $5-10 at: https://platform.openai.com/account/billing")
    print("This project will cost less than $2 total to run.")
    print()

    api_key = input("Enter your OpenAI API key: ")

    if not api_key:
        print("API key required!")
        return None

    # Initialize system with more pages for better coverage
    success = rag_system.initialize_system(api_key, max_pages=15)

    if not success:
        print("Failed to initialize system")
        return None

    print("Enhanced system ready! Now you'll get complete, accurate answers.")
    print("Type 'quit' to exit, 'examples' to see example questions")

    example_questions = [
        "What is the tuition cost for the program?",
        "What scholarships are available for the program?",
        "What are the core courses in the MS program?",
        "What are the admission requirements?",
        "Tell me about the capstone project",
        "What are the application deadlines?",
        "Who are the faculty members?",
        "What career outcomes can I expect?",
        "How long does the program take?",
        "What are the prerequisites for admission?"
    ]

    while True:
        print("\n" + "-"*60)
        user_input = input("Ask a question: ").strip()

        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'examples':
            print("Example Questions:")
            for i, q in enumerate(example_questions, 1):
                print(f"{i}. {q}")
            continue
        elif not user_input:
            continue

        rag_system.ask_question(user_input)

    return rag_system